In [2]:
import os
from sklearn import metrics
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
from masterthesis.data import load_h5ad
from masterthesis.preprocessing import calculate_weights, transform_labels
from sklearn.model_selection import train_test_split
from masterthesis.model_selection import RegularizationGridSearch
from masterthesis.model import SGDBinarizedModel
from sklearn import metrics
import warnings

In [62]:
# report files
genes_outfile = "genes.txt"
results_outfile = "results.txt"

# Fit params
n_seeds = 5
n_folds = 5
n_jobs = 4
n_reg_params = 20
reg_params = np.geomspace(1, 0.005, n_reg_params)
scoring = metrics.make_scorer(metrics.accuracy_score)

# Simulation data
data_dir = "/home/julian/Uni/MasterThesis/data"
filenames = [
    "simdata_TS0.1_SS0.1.h5ad", 
    "simdata_TS0.1_SS0.3.h5ad",
    "simdata_TS0.1_SS0.5.h5ad",
    "simdata_TS0.1_SS0.7.h5ad",
    "simdata_TS0.1_SS0.9.h5ad",
    "simdata_TS0.3_SS0.1.h5ad",
    "simdata_TS0.3_SS0.3.h5ad",
    "simdata_TS0.3_SS0.5.h5ad",
    "simdata_TS0.3_SS0.7.h5ad",
    "simdata_TS0.5_SS0.1.h5ad",
    "simdata_TS0.5_SS0.3.h5ad",
    "simdata_TS0.5_SS0.5.h5ad",
    "simdata_TS0.7_SS0.1.h5ad",
    "simdata_TS0.7_SS0.3.h5ad",
    "simdata_TS0.9_SS0.1.h5ad"
]

In [63]:
warnings.filterwarnings("once")

genes = []
results = {
    "file": [],
    "seed": [],
    "dof": [],
    "train_accuracy": [],
    "train_bal_acc": [],
    "train_abs_err": [],
    "test_accuracy": [],
    "test_bal_acc": [],
    "test_abs_err": [],
    "spearman_corr": [],
    "kendall_corr": [],
    "pearson_corr": []
}

print("[*] Running Simulation")
print("[*] Regularization Params = ", reg_params)

for f in filenames:
    simfile = os.path.join(data_dir, f)
    print("[*] Reading file %s ..." % simfile)
    anndata = load_h5ad(simfile)

    anndata.obs["ordinal_label"] = transform_labels(np.array([int(x) for x in anndata.obs.Ordinal_Time_Labels]))
    X_train, X_test, y_train, y_test = train_test_split(anndata.X, anndata.obs["ordinal_label"], 
                                                        test_size=0.1, 
                                                        stratify=anndata.obs["ordinal_label"],
                                                        random_state=1234)

    weights_train = calculate_weights(y_train)
    weights_test = calculate_weights(y_test)

    for i in range(n_seeds):

        seed = np.random.randint(9999)
        print("... Iteration %s, Seed=%s" % (i, seed))

        print("... Cross Validation")
        sgd = RegularizationGridSearch(estimator=SGDBinarizedModel,
                                       n_folds=n_folds,
                                       n_jobs=n_jobs,
                                       lambdas=reg_params,
                                       scoring=scoring)

        estimator_params = {"random_state": seed}
        fit_params = {"sample_weight": weights_train}
        sgd.fit(X_train, y_train, fit_params=fit_params, estimator_params=estimator_params)

        print("... Refitting on training data")
        sparse_model = sgd.get_optimal_model("1se")
        sparse_model.fit(X_train, y_train)

        # genes weights
        anndata.var["psupertime_weights"] = sparse_model.coef_
        genes += [anndata.var.psupertime_weights[anndata.var.psupertime_weights != 0]]

        # calculate psupertime -> adds anndata.obs.psupertime
        sparse_model.predict_psuper(anndata)
        pearsonr = anndata.obs.Latent_Time.corr(anndata.obs.psupertime)
        spearmanr = anndata.obs.Latent_Time.corr(anndata.obs.psupertime, method='spearman')
        kendalltau = anndata.obs.Latent_Time.corr(anndata.obs.psupertime, method='kendall')

        results["file"] += [f]
        results["seed"] += [seed]
        results["dof"] += [len(np.nonzero(sparse_model.coef_)[0])]
        results["train_accuracy"] += [metrics.accuracy_score(y_train, sparse_model.predict(X_train))]
        results["train_bal_acc"] += [metrics.balanced_accuracy_score(y_train, sparse_model.predict(X_train))]
        results["train_abs_err"] += [metrics.mean_absolute_error(y_train, sparse_model.predict(X_train), sample_weight=weights_train)]
        results["test_accuracy"] += [metrics.accuracy_score(y_test, sparse_model.predict(X_test))]
        results["test_bal_acc"] += [metrics.balanced_accuracy_score(y_test, sparse_model.predict(X_test))]
        results["test_abs_err"] += [metrics.mean_absolute_error(y_test, sparse_model.predict(X_test), sample_weight=weights_test)]
        results["spearman_corr"] += [spearmanr]
        results["kendall_corr"] += [kendalltau]
        results["pearson_corr"] += [pearsonr]

print("[*] Writing results")
# Write results to files
pd.DataFrame(results).to_csv(results_outfile)

with open(genes_outfile, "w") as f:
    for g in genes:#
        if (len(genes) == 0):
            f.write("\n")
        else:
            f.write(", ".join(g.abs().sort_values().index) + "\n")

warnings.filterwarnings("always")


[*] Running Simulation
[*] Regularization Params =  [1.         0.75664673 0.57251427 0.43319105 0.32777259 0.24800806
 0.18765449 0.14198815 0.10743487 0.08129024 0.061508   0.04653982
 0.03521421 0.02664471 0.02016064 0.01525448 0.01154225 0.00873341
 0.0066081  0.005     ]
[*] Reading file /home/julian/Uni/MasterThesis/data/simdata_TS0.1_SS0.1.h5ad ...
... Iteration 0, Seed=7259
... Cross Validation
... Refitting on training data


/home/julian/.local/share/virtualenvs/code-tW9RC7Ez/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


... Iteration 1, Seed=9393
... Cross Validation
... Refitting on training data
... Iteration 2, Seed=8603
... Cross Validation
... Refitting on training data
... Iteration 3, Seed=9557
... Cross Validation
... Refitting on training data


/home/julian/.local/share/virtualenvs/code-tW9RC7Ez/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


... Iteration 4, Seed=2850
... Cross Validation
... Refitting on training data
[*] Reading file /home/julian/Uni/MasterThesis/data/simdata_TS0.1_SS0.3.h5ad ...
... Iteration 0, Seed=3062
... Cross Validation
... Refitting on training data


...

... Iteration 4, Seed=9570
... Cross Validation
... Refitting on training data
[*] Reading file /home/julian/Uni/MasterThesis/data/simdata_TS0.1_SS0.9.h5ad ...
... Iteration 0, Seed=4801
... Cross Validation
... Refitting on training data


/home/julian/.local/share/virtualenvs/code-tW9RC7Ez/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


... Iteration 1, Seed=1788
... Cross Validation
... Refitting on training data
... Iteration 2, Seed=6998
... Cross Validation
... Refitting on training data


/home/julian/.local/share/virtualenvs/code-tW9RC7Ez/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


... Iteration 3, Seed=7169
... Cross Validation
... Refitting on training data
... Iteration 4, Seed=1234
... Cross Validation
... Refitting on training data
[*] Reading file /home/julian/Uni/MasterThesis/data/simdata_TS0.3_SS0.1.h5ad ...
... Iteration 0, Seed=7975
... Cross Validation
... Refitting on training data
... Iteration 1, Seed=5460
... Cross Validation
... Refitting on training data
... Iteration 2, Seed=2215
... Cross Validation
... Refitting on training data
... Iteration 3, Seed=4888
... Cross Validation
... Refitting on training data
... Iteration 4, Seed=4199
... Cross Validation
... Refitting on training data
[*] Reading file /home/julian/Uni/MasterThesis/data/simdata_TS0.3_SS0.3.h5ad ...
... Iteration 0, Seed=4936
... Cross Validation
... Refitting on training data
... Iteration 1, Seed=9583
... Cross Validation
... Refitting on training data
... Iteration 2, Seed=7167
... Cross Validation
... Refitting on training data
... Iteration 3, Seed=9246
... Cross Validation


In [64]:
import datetime
print(datetime.datetime.now())

2023-07-05 02:57:47.599539


**Roughly 6h Runtime for simulation**

In [65]:
import datetime
print(datetime.datetime.now())

2023-07-05 08:35:44.180677


In [67]:
[os.path.join(data_dir, f) for f in filenames]

['/home/julian/Uni/MasterThesis/data/simdata_TS0.1_SS0.1.h5ad',
 '/home/julian/Uni/MasterThesis/data/simdata_TS0.1_SS0.3.h5ad',
 '/home/julian/Uni/MasterThesis/data/simdata_TS0.1_SS0.5.h5ad',
 '/home/julian/Uni/MasterThesis/data/simdata_TS0.1_SS0.7.h5ad',
 '/home/julian/Uni/MasterThesis/data/simdata_TS0.1_SS0.9.h5ad',
 '/home/julian/Uni/MasterThesis/data/simdata_TS0.3_SS0.1.h5ad',
 '/home/julian/Uni/MasterThesis/data/simdata_TS0.3_SS0.3.h5ad',
 '/home/julian/Uni/MasterThesis/data/simdata_TS0.3_SS0.5.h5ad',
 '/home/julian/Uni/MasterThesis/data/simdata_TS0.3_SS0.7.h5ad',
 '/home/julian/Uni/MasterThesis/data/simdata_TS0.5_SS0.1.h5ad',
 '/home/julian/Uni/MasterThesis/data/simdata_TS0.5_SS0.3.h5ad',
 '/home/julian/Uni/MasterThesis/data/simdata_TS0.5_SS0.5.h5ad',
 '/home/julian/Uni/MasterThesis/data/simdata_TS0.7_SS0.1.h5ad',
 '/home/julian/Uni/MasterThesis/data/simdata_TS0.7_SS0.3.h5ad',
 '/home/julian/Uni/MasterThesis/data/simdata_TS0.9_SS0.1.h5ad']